这样做的目的是将每一行（即每个样本）中的最大绝对值缩放为1，其余元素按比例缩放到[-1, 1]之间。

这样可以消除每个样本内部各特征的量纲影响，使得特征值在同一尺度上，便于后续模型训练和收敛。

对于单个样本来说，这样归一化后，特征之间的相对大小关系被保留，且不会因为某一特征数值过大而主导模型。

但对于不同样本之间，如果它们的原始数值范围差异很大，归一化后每一行的最大值都变成1，可能会掩盖样本间的绝对大小差异。

例如，一个样本原始最大值是1000，另一个是1，归一化后都变成最大值1，模型可能无法区分它们的“量级”差别。

因此，这种按行归一化适合于只关心样本内部特征比例、不关心样本间绝对大小的场景。

如果需要保留样本间的绝对大小信息，建议采用全局归一化（如按所有样本的最大/最小值归一化）或标准化等方法。


In [7]:
def normalize(array):
    # 计算每一行的绝对值最大值
    max_number = np.max(np.absolute(array), axis=1, keepdims=True)
    # 防止除以0，如果最大值为0则缩放因子为1，否则为1/最大值
    scaled_rate = np.where(max_number == 0, 1, 1 / max_number)
    # 按行归一化
    norm = array * scaled_rate
    return norm

In [8]:

import numpy as np

# 构造测试数据
test_array = np.array([
    [1, -2, 3],
    [0, 0, 0],
    [-4, 2, -8]
])

# 调用normalize函数
normalized_array = normalize(test_array)

# 打印原始数据和归一化结果
print("原始数据：")
print(test_array)
print("归一化后：")
print(normalized_array)


原始数据：
[[ 1 -2  3]
 [ 0  0  0]
 [-4  2 -8]]
归一化后：
[[ 0.33333333 -0.66666667  1.        ]
 [ 0.          0.          0.        ]
 [-0.5         0.25       -1.        ]]


C:\Users\L\AppData\Local\Temp\ipykernel_70408\1802554402.py:5: RuntimeWarning: divide by zero encountered in divide
  scaled_rate = np.where(max_number == 0, 1, 1 / max_number)


### 以下是归一输出后的神经网络展示


In [9]:
def activation_ReLU(inputs):
    return np.maximum(0,inputs)


In [10]:
def activation_softmax(inputs):
    max_value=np.max(inputs,axis=1,keepdims=True)
    slided_inputs=inputs-max_value
    exp_values=np.exp(slided_inputs)
    norm_base=np.sum(exp_values,axis=1,keepdims=True)
    norm_values=exp_values/norm_base
    return norm_values

In [11]:

class Layer:
    def __init__(self,n_inputs,n_neurons):
        self.weights=np.random.randn(n_inputs,n_neurons) 
        self.biases=np.random.randn(n_neurons)  

    def forward(self,inputs):
        self.sum=np.dot(inputs,self.weights)+self.biases 
        return self.sum  


In [12]:
# normalize（归一化）函数可以在数据预处理阶段用于所有输入数据，也可以在模型输出阶段用于输出结果。  
# 但在神经网络的实现中，通常只在以下两种场景用到：  
# 1. **输入归一化**：在数据进入网络前，先对输入数据进行归一化，有助于模型更快收敛、提升数值稳定性。  
# 2. **输出归一化**：有时在最后一层输出后，为了让输出满足某些范围（如[-1,1]或[0,1]），会对输出再次归一化。  
# 
# 在你当前的Network实现中，normalize只在最后一层输出后用了一次（配合softmax），这是一种常见做法。  
# 如果你希望输入也归一化，可以在数据送入网络前手动调用normalize。  
# 
# 总结：normalize可以用在输入、输出，甚至中间层，具体要看你的任务需求和数据分布。

class Network:
    def __init__(self,network_shape):
        self.shape=network_shape
        self.layers=[]
        for i in range(len(network_shape)-1):
            layer=Layer(network_shape[i],network_shape[i+1])
            self.layers.append(layer)
    #前馈运算
    def network_forward(self,inputs):
        outputs=[inputs]
        for i in range(len(self.layers)):
            layer_output=self.layers[i].forward(outputs[i])
            if i==len(self.layers)-1:
                layer_output=activation_softmax(layer_output)
                layer_output=normalize(layer_output)
            else:
                layer_output=activation_ReLU(layer_output)
            outputs.append(layer_output)
        return outputs

In [13]:
# 测试 Network 类

# 构造一个简单的网络结构，例如输入2维，隐藏层3维，输出2维
network_shape = [2, 3, 2]
net = Network(network_shape)

# 构造一个简单的输入样本（比如2个样本，每个样本2个特征）
X = np.array([[1.0, 2.0],
              [3.0, 4.0]])

# 前向传播
outputs = net.network_forward(X)

# 打印每一层的输出
for idx, out in enumerate(outputs):
    print(f"第{idx}层输出：\n{out}\n")


第0层输出：
[[1. 2.]
 [3. 4.]]

第1层输出：
[[0.         1.78641218 0.86516697]
 [0.90474576 2.38167321 1.98106916]]

第2层输出：
[[9.54814751e-03 1.00000000e+00]
 [9.49830761e-04 1.00000000e+00]]

